# Load Libraries

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2

from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import os
import seaborn as sns

In [2]:
SIZE = 128
data_dir="../../data_extractor/"+str(SIZE)+"/"
out_dir=str(SIZE)+"/"
print(os.listdir(data_dir))
print(os.listdir(out_dir))

['.DS_Store', 'train_images.npy', 'test_images.npy', 'test_labels.npy', 'train_labels.npy']
['.DS_Store']


# Load data arrays

In [3]:
train_images = np.load(data_dir+'/train_images.npy')
test_images = np.load(data_dir+'/test_images.npy')
train_labels= np.load(data_dir+'/train_labels.npy')
test_labels = np.load(data_dir+'/test_labels.npy')

# Encode labels from text to integers.

In [4]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

#pickle the label encoder
np.save(out_dir+'label_encoder.npy', le.classes_)

# Data Pre-processing

In [5]:
# Rename the datasets to proper naming convention
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

# Normalize pixel values to between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

In [6]:
np.save(out_dir+"train_labels", y_train)
np.save(out_dir+"test_labels", y_test)

# Xception

In [7]:
from keras.applications.xception import Xception

# Load model without classifier/fully connected layers
model = Xception(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

# Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in model.layers:
    layer.trainable = False
    
model.summary()  #Trainable parameters will be 0

#Send train data from feature extractor process
feature_extractor=model.predict(x_train)
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

#Send test data through same feature extractor process
X_test_feature = model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

np.save(out_dir+"Xception_train_images", features)
np.save(out_dir+"Xception_test_images", X_test_features)

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 63, 63, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 63, 63, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 63, 63, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

# VGG 16

In [8]:
from keras.applications.vgg16 import VGG16

# Load model without classifier/fully connected layers
model = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

# Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in model.layers:
    layer.trainable = False
    
print(model.summary())  #Trainable parameters will be 0

#Send train data from feature extractor process
feature_extractor=model.predict(x_train)
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

#Send test data through same feature extractor process
X_test_feature = model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

np.save(out_dir+"VGG16_train_images", features)
np.save(out_dir+"VGG16_test_images", X_test_features)

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

# VGG19

In [9]:
from keras.applications.vgg19 import VGG19

# Load model without classifier/fully connected layers
model = VGG19(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

# Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in model.layers:
    layer.trainable = False
    
print(model.summary())  #Trainable parameters will be 0

#Send train data from feature extractor process
feature_extractor=model.predict(x_train)
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

#Send test data through same feature extractor process
X_test_feature = model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

np.save(out_dir+"VGG19_train_images", features)
np.save(out_dir+"VGG19_test_images", X_test_features)

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

# ResNet50

In [10]:
from keras.applications.resnet50 import ResNet50

# Load model without classifier/fully connected layers
model = ResNet50(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

# Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in model.layers:
    layer.trainable = False
    
print(model.summary())  #Trainable parameters will be 0

#Send train data from feature extractor process
feature_extractor=model.predict(x_train)
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

#Send test data through same feature extractor process
X_test_feature = model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

np.save(out_dir+"ResNet50_train_images", features)
np.save(out_dir+"ResNet50_test_images", X_test_features)

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

# ResNet101

In [11]:
from keras.applications.resnet import ResNet101

# Load model without classifier/fully connected layers
model = ResNet101(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

# Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in model.layers:
    layer.trainable = False
    
print(model.summary())  #Trainable parameters will be 0

#Send train data from feature extractor process
feature_extractor=model.predict(x_train)
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

#Send test data through same feature extractor process
X_test_feature = model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

np.save(out_dir+"ResNet101_train_images", features)
np.save(out_dir+"ResNet101_test_images", X_test_features)

Model: "resnet101"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1_conv[0][0]                 
__________________________________________________________________________________________

# ResNet152

In [12]:
from keras.applications.resnet import ResNet152

# Load model without classifier/fully connected layers
model = ResNet152(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

# Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in model.layers:
    layer.trainable = False
    
print(model.summary())  #Trainable parameters will be 0

#Send train data from feature extractor process
feature_extractor=model.predict(x_train)
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

#Send test data through same feature extractor process
X_test_feature = model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

np.save(out_dir+"ResNet152_train_images", features)
np.save(out_dir+"ResNet152_test_images", X_test_features)

Model: "resnet152"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1_conv[0][0]                 
__________________________________________________________________________________________

# ResNet50V2

In [13]:
from keras.applications.resnet_v2 import ResNet50V2

# Load model without classifier/fully connected layers
model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

# Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in model.layers:
    layer.trainable = False
    
print(model.summary())  #Trainable parameters will be 0

#Send train data from feature extractor process
feature_extractor=model.predict(x_train)
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

#Send test data through same feature extractor process
X_test_feature = model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

np.save(out_dir+"ResNet50V2_train_images", features)
np.save(out_dir+"ResNet50V2_test_images", X_test_features)

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 66, 66, 64)   0           conv1_conv[0][0]                 
_________________________________________________________________________________________

# ResNet101V2

In [14]:
from keras.applications.resnet_v2 import ResNet101V2

# Load model without classifier/fully connected layers
model = ResNet101V2(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

# Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in model.layers:
    layer.trainable = False
    
print(model.summary())  #Trainable parameters will be 0

#Send train data from feature extractor process
feature_extractor=model.predict(x_train)
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

#Send test data through same feature extractor process
X_test_feature = model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

np.save(out_dir+"ResNet101V2_train_images", features)
np.save(out_dir+"ResNet101V2_test_images", X_test_features)

Model: "resnet101v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_8[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 66, 66, 64)   0           conv1_conv[0][0]                 
________________________________________________________________________________________

# ResNet152V2

In [15]:
from keras.applications.resnet_v2 import ResNet152V2

# Load model without classifier/fully connected layers
model = ResNet152V2(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

# Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in model.layers:
    layer.trainable = False
    
print(model.summary())  #Trainable parameters will be 0

#Send train data from feature extractor process
feature_extractor=model.predict(x_train)
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

#Send test data through same feature extractor process
X_test_feature = model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

np.save(out_dir+"ResNet152V2_train_images", features)
np.save(out_dir+"ResNet152V2_test_images", X_test_features)

Model: "resnet152v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_9[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 66, 66, 64)   0           conv1_conv[0][0]                 
________________________________________________________________________________________

# InceptionV3

In [16]:
from keras.applications.inception_v3 import InceptionV3

# Load model without classifier/fully connected layers
model = InceptionV3(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

# Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in model.layers:
    layer.trainable = False
    
print(model.summary())  #Trainable parameters will be 0

#Send train data from feature extractor process
feature_extractor=model.predict(x_train)
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

#Send test data through same feature extractor process
X_test_feature = model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

np.save(out_dir+"InceptionV3_train_images", features)
np.save(out_dir+"InceptionV3_test_images", X_test_features)

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 63, 63, 32)   864         input_10[0][0]                   
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 63, 63, 32)   96          conv2d_4[0][0]                   
__________________________________________________________________________________________________
activation (Activation)         (None, 63, 63, 32)   0           batch_normalization_4[0][0]      
_______________________________________________________________________________________

# InceptionResNetV2

In [17]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2

# Load model without classifier/fully connected layers
model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

# Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in model.layers:
    layer.trainable = False
    
print(model.summary())  #Trainable parameters will be 0

#Send train data from feature extractor process
feature_extractor=model.predict(x_train)
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

#Send test data through same feature extractor process
X_test_feature = model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

np.save(out_dir+"InceptionResNetV2_train_images", features)
np.save(out_dir+"InceptionResNetV2_test_images", X_test_features)

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_98 (Conv2D)              (None, 63, 63, 32)   864         input_11[0][0]                   
__________________________________________________________________________________________________
batch_normalization_98 (BatchNo (None, 63, 63, 32)   96          conv2d_98[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 63, 63, 32)   0           batch_normalization_98[0][0]     
________________________________________________________________________________

__________________________________________________________________________________________________
block8_8_ac (Activation)        (None, 2, 2, 2080)   0           block8_8[0][0]                   
__________________________________________________________________________________________________
conv2d_294 (Conv2D)             (None, 2, 2, 192)    399360      block8_8_ac[0][0]                
__________________________________________________________________________________________________
batch_normalization_294 (BatchN (None, 2, 2, 192)    576         conv2d_294[0][0]                 
__________________________________________________________________________________________________
activation_290 (Activation)     (None, 2, 2, 192)    0           batch_normalization_294[0][0]    
__________________________________________________________________________________________________
conv2d_295 (Conv2D)             (None, 2, 2, 224)    129024      activation_290[0][0]             
__________

# MobileNet

In [18]:
from keras.applications.mobilenet import MobileNet

# Load model without classifier/fully connected layers
model = MobileNet(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

# Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in model.layers:
    layer.trainable = False
    
print(model.summary())  #Trainable parameters will be 0

#Send train data from feature extractor process
feature_extractor=model.predict(x_train)
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

#Send test data through same feature extractor process
X_test_feature = model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

np.save(out_dir+"MobileNet_train_images", features)
np.save(out_dir+"MobileNet_test_images", X_test_features)

17227776/17225924 [==============================] - 4s 0us/step
Model: "mobilenet_1.00_128"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 129, 129, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
_________________________________________________

# MobileNetV2

In [19]:
from keras.applications.mobilenet_v2 import MobileNetV2

# Load model without classifier/fully connected layers
model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

# Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in model.layers:
    layer.trainable = False
    
print(model.summary())  #Trainable parameters will be 0

#Send train data from feature extractor process
feature_extractor=model.predict(x_train)
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

#Send test data through same feature extractor process
X_test_feature = model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

np.save(out_dir+"MobileNetV2_train_images", features)
np.save(out_dir+"MobileNetV2_test_images", X_test_features)

9412608/9406464 [==============================] - 2s 0us/step
Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_13[0][0]                   
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
________________

block_11_add (Add)              (None, 8, 8, 96)     0           block_10_project_BN[0][0]        
                                                                 block_11_project_BN[0][0]        
__________________________________________________________________________________________________
block_12_expand (Conv2D)        (None, 8, 8, 576)    55296       block_11_add[0][0]               
__________________________________________________________________________________________________
block_12_expand_BN (BatchNormal (None, 8, 8, 576)    2304        block_12_expand[0][0]            
__________________________________________________________________________________________________
block_12_expand_relu (ReLU)     (None, 8, 8, 576)    0           block_12_expand_BN[0][0]         
__________________________________________________________________________________________________
block_12_depthwise (DepthwiseCo (None, 8, 8, 576)    5184        block_12_expand_relu[0][0]       
__________

# DenseNet121

In [20]:
from keras.applications.densenet import DenseNet121

# Load model without classifier/fully connected layers
model = DenseNet121(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

# Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in model.layers:
    layer.trainable = False
    
print(model.summary())  #Trainable parameters will be 0

#Send train data from feature extractor process
feature_extractor=model.predict(x_train)
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

#Send test data through same feature extractor process
X_test_feature = model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

np.save(out_dir+"DenseNet121_train_images", features)
np.save(out_dir+"DenseNet121_test_images", X_test_features)

Model: "densenet121"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 134, 134, 3)  0           input_14[0][0]                   
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 64, 64, 64)   9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1/conv[0][0]                 
________________________________________________________________________________________

# DenseNet169

In [21]:
from keras.applications.densenet import DenseNet169

# Load model without classifier/fully connected layers
model = DenseNet169(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

# Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in model.layers:
    layer.trainable = False
    
print(model.summary())  #Trainable parameters will be 0

#Send train data from feature extractor process
feature_extractor=model.predict(x_train)
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

#Send test data through same feature extractor process
X_test_feature = model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

np.save(out_dir+"DenseNet169_train_images", features)
np.save(out_dir+"DenseNet169_test_images", X_test_features)

Model: "densenet169"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 134, 134, 3)  0           input_15[0][0]                   
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 64, 64, 64)   9408        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1/conv[0][0]                 
________________________________________________________________________________________

# DenseNet201

In [22]:
from keras.applications.densenet import DenseNet201

# Load model without classifier/fully connected layers
model = DenseNet201(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

# Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in model.layers:
    layer.trainable = False
    
print(model.summary())  #Trainable parameters will be 0

#Send train data from feature extractor process
feature_extractor=model.predict(x_train)
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

#Send test data through same feature extractor process
X_test_feature = model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

np.save(out_dir+"DenseNet201_train_images", features)
np.save(out_dir+"DenseNet201_test_images", X_test_features)

Model: "densenet201"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_4 (ZeroPadding2D (None, 134, 134, 3)  0           input_16[0][0]                   
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 64, 64, 64)   9408        zero_padding2d_4[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1/conv[0][0]                 
________________________________________________________________________________________

Total params: 18,321,984
Trainable params: 0
Non-trainable params: 18,321,984
__________________________________________________________________________________________________
None
